In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from random import randint
from fake_useragent import UserAgent
import concurrent.futures
from tqdm import notebook
from itertools import repeat
import tqdm



In [2]:
crawl_data = pd.read_csv('user_review_v1_MOBIUZ.csv',encoding='utf-8')
print(len(crawl_data))

4826


In [3]:
user_agent = UserAgent()
ua = user_agent

In [4]:
proxy_list=['45.63.19.165:80','34.134.60.185:443','184.177.84.245:1088','50.250.205.21:32100','67.201.33.10:25283','71.71.162.234:39593','185.59.66.210:33611','181.209.219.28:4153','64.124.191.98:32688',
'12.216.103.114:39072','38.133.200.94:31596','174.108.154.129:32940','70.60.230.8:34116','170.106.155.14:1080','181.129.144.59:3629']
test_proxy = {f'{proxy}':0 for proxy in proxy_list}
print(test_proxy)
def proxy(idx):
    proxies = {"http":"socks4://"+str(proxy_list[idx%len(proxy_list)])}
    return proxies
    

{'45.63.19.165:80': 0, '34.134.60.185:443': 0, '184.177.84.245:1088': 0, '50.250.205.21:32100': 0, '67.201.33.10:25283': 0, '71.71.162.234:39593': 0, '185.59.66.210:33611': 0, '181.209.219.28:4153': 0, '64.124.191.98:32688': 0, '12.216.103.114:39072': 0, '38.133.200.94:31596': 0, '174.108.154.129:32940': 0, '70.60.230.8:34116': 0, '170.106.155.14:1080': 0, '181.129.144.59:3629': 0}


In [5]:

def get_cookies(asin):
    url = str('https://www.amazon.com/dp/'+str(asin)+'?th=1')
    headers={'User-agent':ua.random}
    res = requests.get(url,headers = headers)
    cookiejar = res.cookies
    return cookiejar

#get_cookies(crawl_data['Asin'][randint(0,50)])
    


In [6]:
failed_asin=[]

In [26]:
def request_web(url,idx):
    headers={'User-agent':ua.random,
        'Cookie':str(get_cookies(crawl_data['Asin'][randint(0,50)]))}
    prox = proxy(idx)
    try:
        response = requests.get(url,headers=headers,proxies=prox)
        txt = response.text
        return txt
    except:
        #test_proxy[str(f'{prox}')] += 1
        failed_asin.append(url)
        print('Requests Failed')
        return False
    
        


In [8]:
def get_content(soap):
    category = soap.find_all("a",class_="a-link-normal a-color-tertiary")
    price = soap.find("span",class_='a-offscreen')
    try:
        brand = soap.find(text='Brand').findNext('td').text
    except:
        brand = 'None'
    if category != []:
        category = [str(t).split('">')[1].split('</a>')[0].replace(" ","").replace('\n',"").replace('&amp;',"") for t in category]
        try:
            price = str(price).split('$')[1].replace('</span>','')
        except:
            price = 'Non-Available'
        content_dict={'category':category,'price':price,'Brand':brand}
        return content_dict
    else:
        #print('Content Failed')
        return False
        
    

In [13]:

def scrape(arg):
    url,idx = arg
    tmp_format = pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    if request_web(url,idx) != False:
        soap = BeautifulSoup(request_web(url,idx),"lxml")
        if get_content(soap) != False:
            brand = get_content(soap)['Brand']
            category = get_content(soap)['category']
            price = get_content(soap)['price']
            tmp_format = tmp_format.append({
            'Asin':url.split('/dp/')[1],'Category':[category[1]],'Sub_category':[category[-2]],
            'Breakdown_category':category[-1],'Price':[price],'Brand':[brand]},ignore_index=True)
            time.sleep(3)
        else:
            failed_asin.append(url)
            time.sleep(2)
                
    return tmp_format


In [10]:
failed_asin=[]
final_format=pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})

In [11]:
def run():
    base_url = 'https://www.amazon.com'
    url_list=[f"{base_url}/dp/{asin}" for asin in crawl_data['Asin']]
    tmp_format = pd.DataFrame({'Asin':[],'Brand':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    idx_list=[]
    for idx in range(len(crawl_data)-1):
        idx_list.append(idx)
    arg = list(zip(url_list,idx_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
        for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):
           tmp_format = tmp_format.append(result,ignore_index = True)
        executor.shutdown()
    return tmp_format
    


    

In [15]:
final_format = run()
print('__________________RESULT_____________________')
print('失敗數:'+str(len(failed_asin)),
      'Proxy error:'+str(test_proxy),
      final_format,sep='\n')

/var/folders/9f/cbry14m53b7__fd2jstjs52h0000gn/T/ipykernel_45020/3625239620.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):


  0%|          | 0/4826 [00:00<?, ?it/s]

__________________RESULT_____________________
失敗數:1801
Proxy error:{'45.63.19.165:80': 0, '34.134.60.185:443': 0, '184.177.84.245:1088': 0, '50.250.205.21:32100': 0, '67.201.33.10:25283': 0, '71.71.162.234:39593': 0, '185.59.66.210:33611': 0, '181.209.219.28:4153': 0, '64.124.191.98:32688': 0, '12.216.103.114:39072': 0, '38.133.200.94:31596': 0, '174.108.154.129:32940': 0, '70.60.230.8:34116': 0, '170.106.155.14:1080': 0, '181.129.144.59:3629': 0}
            Asin       Brand                Category  \
0     B01DUB4GVO      [None]     [OfficeElectronics]   
1     B07ZR5DSMQ      [None]     [HouseholdSupplies]   
2     B08DP4SVC7      [None]   [LightingCeilingFans]   
3     B07ZQM54HP   [ Geyes ]  [ComputersAccessories]   
4     B08L7T8RSP  [ Deconn ]  [ComputersAccessories]   
...          ...         ...                     ...   
3154  B07PY7GLKV      [None]   [LightingCeilingFans]   
3155  B08FF6DJNP    [ BenQ ]  [ComputersAccessories]   
3156  B07DFFFNWL   [ KTRIO ]  [OfficeSchoolS

In [16]:
final_format.to_csv('Asin_Category_MOBIUZ_list1.csv')

#### Retry failed URLs in first crawling 
##### Retry Loop until it done

In [34]:
print(len(failed_asin))

4978


In [18]:
failed_format = pd.DataFrame({'fail':failed_asin})
failed_format.to_csv('failed_list.csv')

In [19]:
failed2_asin=pd.read_csv('failed_list.csv')['fail']
failed2_asin

0       https://www.amazon.com/dp/B08C74694Z
1       https://www.amazon.com/dp/B01LR3RAVU
2       https://www.amazon.com/dp/B081RFK5JM
3             https://www.amazon.com/dp/None
4       https://www.amazon.com/dp/B08L7T8RSP
                        ...                 
1796    https://www.amazon.com/dp/B08FM9KRSZ
1797    https://www.amazon.com/dp/B075ZWHNJT
1798    https://www.amazon.com/dp/B06XW7MLB4
1799    https://www.amazon.com/dp/B0182UTI5W
1800    https://www.amazon.com/dp/B07NQVV385
Name: fail, Length: 1801, dtype: object

In [20]:
failed_asin=[]

In [21]:
def fail_run():
    url_list=[asin for asin in failed2_asin]
    tmp_format = pd.DataFrame({'Asin':[],'Category':[],'Sub_category':[],'Breakdown_category':[],'Price':[]})
    idx_list=[]
    for idx in range(len(failed2_asin)-1):
        idx_list.append(idx)
    arg = list(zip(url_list,idx_list))
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):
           tmp_format = tmp_format.append(result,ignore_index = True)
        executor.shutdown()
    return tmp_format

In [28]:
retry_format = fail_run()
print('__________________RESULT_____________________')
print('失敗數:'+str(len(failed_asin)),
      'Proxy error:'+str(test_proxy),
      retry_format,sep='\n')


/var/folders/9f/cbry14m53b7__fd2jstjs52h0000gn/T/ipykernel_45020/1848244947.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for result in tqdm.tqdm_notebook((executor.map(scrape,arg)),total=len(url_list)):


  0%|          | 0/1801 [00:00<?, ?it/s]

__________________RESULT_____________________
失敗數:4978
Proxy error:{'45.63.19.165:80': 0, '34.134.60.185:443': 0, '184.177.84.245:1088': 0, '50.250.205.21:32100': 0, '67.201.33.10:25283': 0, '71.71.162.234:39593': 0, '185.59.66.210:33611': 0, '181.209.219.28:4153': 0, '64.124.191.98:32688': 0, '12.216.103.114:39072': 0, '38.133.200.94:31596': 0, '174.108.154.129:32940': 0, '70.60.230.8:34116': 0, '170.106.155.14:1080': 0, '181.129.144.59:3629': 0}
           Asin                Category                  Sub_category  \
0    B01LR3RAVU   [LightingCeilingFans]             [NoveltyLighting]   
1    B081RFK5JM             [Furniture]         [HomeOfficeFurniture]   
2    B08L7T8RSP  [ComputersAccessories]       [ComputerCableAdapters]   
3    B07XBJR7GY   [LightingCeilingFans]             [NoveltyLighting]   
4    B07HSFN9W2          [PlayStation4]                 [Accessories]   
..          ...                     ...                           ...   
128  B077VXV323  [ComputersAccessorie

In [29]:
retry_format

,Asin,Category,Sub_category,Breakdown_category,Price,Brand
0,B01LR3RAVU,[LightingCeilingFans],[NoveltyLighting],LEDStripLights,[16.99],[ Power Practical ]
1,B081RFK5JM,[Furniture],[HomeOfficeFurniture],HomeOfficeDesks,[Non-Available],[ NSdirect ]
2,B08L7T8RSP,[ComputersAccessories],[ComputerCableAdapters],DVI-HDMIAdapters,[11.99],[ Deconn ]
3,B07XBJR7GY,[LightingCeilingFans],[NoveltyLighting],LEDStripLights,[25.98],[ Nexillumi ]
4,B07HSFN9W2,[PlayStation4],[Accessories],Headsets,[249.99],[ SteelSeries ]
...,...,...,...,...,...,...
128,B077VXV323,[ComputersAccessories],[DataStorage],USBFlashDrives,[7.53],[ SanDisk ]
129,B07W3QM1YR,[NintendoSwitch],"[Faceplates,ProtectorsSkins]",ScreenProtectors,[Non-Available],[None]
130,B078VM9RVB,[NoveltyMore],[HatsCaps],BaseballCaps,[16.98],[None]
131,B0841JFFCR,[ComputersAccessories],"[Keyboards,MiceAccessories]",Keyboards,[42.99],[ HK GAMING ]


In [30]:
retry_format.to_csv('Asin_Category_retry.csv')

#### Concat 1st & 2nd Crawling result

In [31]:
one = pd.read_csv('Asin_Category_MOBIUZ_list1.csv')
retry= pd.read_csv('Asin_Category_retry.csv')


In [32]:
concat_all = pd.concat([one,retry],ignore_index = True)
print(concat_all)

      Unnamed: 0        Asin                Brand                  Category  \
0              0  B01DUB4GVO             ['None']     ['OfficeElectronics']   
1              1  B07ZR5DSMQ             ['None']     ['HouseholdSupplies']   
2              2  B08DP4SVC7             ['None']   ['LightingCeilingFans']   
3              3  B07ZQM54HP          [' Geyes ']  ['ComputersAccessories']   
4              4  B08L7T8RSP         [' Deconn ']  ['ComputersAccessories']   
...          ...         ...                  ...                       ...   
3287         128  B077VXV323        [' SanDisk ']  ['ComputersAccessories']   
3288         129  B07W3QM1YR             ['None']        ['NintendoSwitch']   
3289         130  B078VM9RVB             ['None']           ['NoveltyMore']   
3290         131  B0841JFFCR      [' HK GAMING ']  ['ComputersAccessories']   
3291         132  B075ZWHNJT  [' Amazon Basics ']   ['AccessoriesSupplies']   

                        Sub_category           Brea

In [33]:
concat_all.to_csv('Asin_Category_MOBIUZ_list.csv')